두번째 셀의 3번줄이 수집시간인데 720분으로 설정되어있어서 매일 아침11시에 실행시키시고, 오후 11시에 코드가 종료될겁니다

셀 실행전 매일 수정해주셔야 될것 하나:
두번째 셀에서 밑에서 두번째 줄에 저장파일명을 바꿔줘야 되는데 7000_요일.csv로 바꿔 주심 됩니다~

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import warnings
import time
import sklearn.preprocessing

pd.set_option("display.max_columns", None)
warnings.filterwarnings(action='ignore')

serviceKey='bwUUK6MEA6GnqDpo2ZS9ChN4VZ1nkUPXrryU80XxUcLqxB62pIGIPWXNdyzqgT20BHrw8zxWWk9rqiSQwTrbnQ=='

p7000 ={'serviceKey' :serviceKey,
             'routeId':'200000112','stationId':'228000703'}
class Bus():
    def __init__(self,params):
        self.params=params
        self.bus_time=[]
        
    def repet(self):
        url = 'http://apis.data.go.kr/6410000/busarrivalservice/getBusArrivalItem'
        response = requests.get(url, params=self.params)
        soup=BeautifulSoup(response.text,'xml')
        return soup

        
    def current_buses(self): 
        url = 'http://apis.data.go.kr/6410000/buslocationservice/getBusLocationList'
        mod_params ={'serviceKey' : self.params['serviceKey'],
                  'routeId':self.params['routeId']}

        response=requests.get(url,params=mod_params)
        soup=BeautifulSoup(response.text,'xml')
        locations=sorted([int(x.stationSeq.text) for x in soup.find_all('busLocationList')])
        return locations


def trim_data(df,FileName):
    df['시간'] = df['시간'].apply(pd.to_datetime)
    df['hour']=df['시간'].dt.hour
    df['minute']=df['시간'].dt.minute
    df['second']=df['시간'].dt.second
    df['day'] = df['시간'].dt.weekday 

    mb = sklearn.preprocessing.MultiLabelBinarizer()
    if isinstance(df['운행중인 다른 버스들의 위치'].iloc[0], str):
        df['운행중인 다른 버스들의 위치'] = df['운행중인 다른 버스들의 위치'].apply(eval)
    entity = mb.fit_transform(df['운행중인 다른 버스들의 위치']) 
    lsts=pd.DataFrame(entity,columns=mb.classes_)
    lsts.columns=[f'station{x}' for x in lsts.columns]
    df=pd.concat([df,lsts],axis=1,sort=False).reset_index(drop=True)


    df=df.drop(["운행중인 다른 버스들의 위치"],axis=1)
    
    batch=df.index[df.arrived==1]
    df=df.iloc[:batch[-1]+1,:]
    #7/12 -----------------------------------
    batch=df.index[df.arrived==1]
    temp=0
    for x in batch:
        for y in range(temp,x): 
            df['arrived'][y]=(df['시간'][x]-df['시간'][y]).seconds
            temp=x+1
    for e in batch:
        df['arrived'][e]=0
    # 7/12 -----------------------------------

    df = df.set_index('시간')
    df.to_csv(f'./{FileName}')        # 동일 폴더에 파일이 저장될 거임
    return df

In [ ]:
past=4
pBus=Bus(p7000)
for x in range(720): # 시간 설정
    soup=pBus.repet()
    try:
        result=int(soup.resultCode.text)
        arrived=0
        if past!=result:
            if result==0:
                arrived=1
        pBus.bus_time.append([soup.queryTime.text,pBus.current_buses(),arrived])
        past=result
        print(x,"번째 시도: ",pBus.bus_time)
        time.sleep(60)
    except Exception as e:
        continue

df=pd.DataFrame(pBus.bus_time,columns=['시간','운행중인 다른 버스들의 위치','arrived'])

mod_df=trim_data(df,'7000_mon.csv') # 이름 설정
mod_df

0 번째 시도:  [['2022-08-01 12:38:55.653', [8, 14, 26, 31, 40, 47, 60, 70], 0]]
1 번째 시도:  [['2022-08-01 12:38:55.653', [8, 14, 26, 31, 40, 47, 60, 70], 0], ['2022-08-01 12:39:55.767', [8, 16, 26, 32, 40, 47, 61, 71], 0]]
2 번째 시도:  [['2022-08-01 12:38:55.653', [8, 14, 26, 31, 40, 47, 60, 70], 0], ['2022-08-01 12:39:55.767', [8, 16, 26, 32, 40, 47, 61, 71], 0], ['2022-08-01 12:40:55.897', [9, 16, 26, 33, 41, 48, 62, 71], 0]]
3 번째 시도:  [['2022-08-01 12:38:55.653', [8, 14, 26, 31, 40, 47, 60, 70], 0], ['2022-08-01 12:39:55.767', [8, 16, 26, 32, 40, 47, 61, 71], 0], ['2022-08-01 12:40:55.897', [9, 16, 26, 33, 41, 48, 62, 71], 0], ['2022-08-01 12:41:55.996', [9, 16, 27, 33, 41, 48, 63, 72], 0]]
4 번째 시도:  [['2022-08-01 12:38:55.653', [8, 14, 26, 31, 40, 47, 60, 70], 0], ['2022-08-01 12:39:55.767', [8, 16, 26, 32, 40, 47, 61, 71], 0], ['2022-08-01 12:40:55.897', [9, 16, 26, 33, 41, 48, 62, 71], 0], ['2022-08-01 12:41:55.996', [9, 16, 27, 33, 41, 48, 63, 72], 0], ['2022-08-01 12:42:56.252', [9, 17,

17 번째 시도:  [['2022-08-01 12:38:55.653', [8, 14, 26, 31, 40, 47, 60, 70], 0], ['2022-08-01 12:39:55.767', [8, 16, 26, 32, 40, 47, 61, 71], 0], ['2022-08-01 12:40:55.897', [9, 16, 26, 33, 41, 48, 62, 71], 0], ['2022-08-01 12:41:55.996', [9, 16, 27, 33, 41, 48, 63, 72], 0], ['2022-08-01 12:42:56.252', [9, 17, 27, 33, 41, 49, 63, 73], 0], ['2022-08-01 12:43:56.400', [10, 18, 27, 34, 43, 49, 63, 73], 0], ['2022-08-01 12:44:56.906', [11, 18, 28, 35, 44, 50, 63, 75], 0], ['2022-08-01 12:45:57.021', [11, 18, 28, 36, 45, 50, 65, 75], 0], ['2022-08-01 12:46:57.137', [11, 19, 29, 37, 45, 51, 65, 75], 0], ['2022-08-01 12:47:57.408', [12, 20, 29, 37, 46, 51, 65, 76], 0], ['2022-08-01 12:48:57.805', [13, 21, 29, 38, 46, 51, 66, 77], 0], ['2022-08-01 12:49:57.920', [13, 21, 30, 39, 47, 51, 66, 79], 0], ['2022-08-01 12:50:58.066', [1, 14, 21, 31, 40, 48, 52, 68, 79], 1], ['2022-08-01 12:51:58.228', [3, 15, 22, 31, 40, 48, 52, 68], 0], ['2022-08-01 12:52:58.374', [4, 15, 23, 32, 41, 48, 52, 68], 0], ['

24 번째 시도:  [['2022-08-01 12:38:55.653', [8, 14, 26, 31, 40, 47, 60, 70], 0], ['2022-08-01 12:39:55.767', [8, 16, 26, 32, 40, 47, 61, 71], 0], ['2022-08-01 12:40:55.897', [9, 16, 26, 33, 41, 48, 62, 71], 0], ['2022-08-01 12:41:55.996', [9, 16, 27, 33, 41, 48, 63, 72], 0], ['2022-08-01 12:42:56.252', [9, 17, 27, 33, 41, 49, 63, 73], 0], ['2022-08-01 12:43:56.400', [10, 18, 27, 34, 43, 49, 63, 73], 0], ['2022-08-01 12:44:56.906', [11, 18, 28, 35, 44, 50, 63, 75], 0], ['2022-08-01 12:45:57.021', [11, 18, 28, 36, 45, 50, 65, 75], 0], ['2022-08-01 12:46:57.137', [11, 19, 29, 37, 45, 51, 65, 75], 0], ['2022-08-01 12:47:57.408', [12, 20, 29, 37, 46, 51, 65, 76], 0], ['2022-08-01 12:48:57.805', [13, 21, 29, 38, 46, 51, 66, 77], 0], ['2022-08-01 12:49:57.920', [13, 21, 30, 39, 47, 51, 66, 79], 0], ['2022-08-01 12:50:58.066', [1, 14, 21, 31, 40, 48, 52, 68, 79], 1], ['2022-08-01 12:51:58.228', [3, 15, 22, 31, 40, 48, 52, 68], 0], ['2022-08-01 12:52:58.374', [4, 15, 23, 32, 41, 48, 52, 68], 0], ['

29 번째 시도:  [['2022-08-01 12:38:55.653', [8, 14, 26, 31, 40, 47, 60, 70], 0], ['2022-08-01 12:39:55.767', [8, 16, 26, 32, 40, 47, 61, 71], 0], ['2022-08-01 12:40:55.897', [9, 16, 26, 33, 41, 48, 62, 71], 0], ['2022-08-01 12:41:55.996', [9, 16, 27, 33, 41, 48, 63, 72], 0], ['2022-08-01 12:42:56.252', [9, 17, 27, 33, 41, 49, 63, 73], 0], ['2022-08-01 12:43:56.400', [10, 18, 27, 34, 43, 49, 63, 73], 0], ['2022-08-01 12:44:56.906', [11, 18, 28, 35, 44, 50, 63, 75], 0], ['2022-08-01 12:45:57.021', [11, 18, 28, 36, 45, 50, 65, 75], 0], ['2022-08-01 12:46:57.137', [11, 19, 29, 37, 45, 51, 65, 75], 0], ['2022-08-01 12:47:57.408', [12, 20, 29, 37, 46, 51, 65, 76], 0], ['2022-08-01 12:48:57.805', [13, 21, 29, 38, 46, 51, 66, 77], 0], ['2022-08-01 12:49:57.920', [13, 21, 30, 39, 47, 51, 66, 79], 0], ['2022-08-01 12:50:58.066', [1, 14, 21, 31, 40, 48, 52, 68, 79], 1], ['2022-08-01 12:51:58.228', [3, 15, 22, 31, 40, 48, 52, 68], 0], ['2022-08-01 12:52:58.374', [4, 15, 23, 32, 41, 48, 52, 68], 0], ['

34 번째 시도:  [['2022-08-01 12:38:55.653', [8, 14, 26, 31, 40, 47, 60, 70], 0], ['2022-08-01 12:39:55.767', [8, 16, 26, 32, 40, 47, 61, 71], 0], ['2022-08-01 12:40:55.897', [9, 16, 26, 33, 41, 48, 62, 71], 0], ['2022-08-01 12:41:55.996', [9, 16, 27, 33, 41, 48, 63, 72], 0], ['2022-08-01 12:42:56.252', [9, 17, 27, 33, 41, 49, 63, 73], 0], ['2022-08-01 12:43:56.400', [10, 18, 27, 34, 43, 49, 63, 73], 0], ['2022-08-01 12:44:56.906', [11, 18, 28, 35, 44, 50, 63, 75], 0], ['2022-08-01 12:45:57.021', [11, 18, 28, 36, 45, 50, 65, 75], 0], ['2022-08-01 12:46:57.137', [11, 19, 29, 37, 45, 51, 65, 75], 0], ['2022-08-01 12:47:57.408', [12, 20, 29, 37, 46, 51, 65, 76], 0], ['2022-08-01 12:48:57.805', [13, 21, 29, 38, 46, 51, 66, 77], 0], ['2022-08-01 12:49:57.920', [13, 21, 30, 39, 47, 51, 66, 79], 0], ['2022-08-01 12:50:58.066', [1, 14, 21, 31, 40, 48, 52, 68, 79], 1], ['2022-08-01 12:51:58.228', [3, 15, 22, 31, 40, 48, 52, 68], 0], ['2022-08-01 12:52:58.374', [4, 15, 23, 32, 41, 48, 52, 68], 0], ['

...

325 번째 시도:  [['2022-08-01 12:38:55.653', [8, 14, 26, 31, 40, 47, 60, 70], 0], ['2022-08-01 12:39:55.767', [8, 16, 26, 32, 40, 47, 61, 71], 0], ['2022-08-01 12:40:55.897', [9, 16, 26, 33, 41, 48, 62, 71], 0], ['2022-08-01 12:41:55.996', [9, 16, 27, 33, 41, 48, 63, 72], 0], ['2022-08-01 12:42:56.252', [9, 17, 27, 33, 41, 49, 63, 73], 0], ['2022-08-01 12:43:56.400', [10, 18, 27, 34, 43, 49, 63, 73], 0], ['2022-08-01 12:44:56.906', [11, 18, 28, 35, 44, 50, 63, 75], 0], ['2022-08-01 12:45:57.021', [11, 18, 28, 36, 45, 50, 65, 75], 0], ['2022-08-01 12:46:57.137', [11, 19, 29, 37, 45, 51, 65, 75], 0], ['2022-08-01 12:47:57.408', [12, 20, 29, 37, 46, 51, 65, 76], 0], ['2022-08-01 12:48:57.805', [13, 21, 29, 38, 46, 51, 66, 77], 0], ['2022-08-01 12:49:57.920', [13, 21, 30, 39, 47, 51, 66, 79], 0], ['2022-08-01 12:50:58.066', [1, 14, 21, 31, 40, 48, 52, 68, 79], 1], ['2022-08-01 12:51:58.228', [3, 15, 22, 31, 40, 48, 52, 68], 0], ['2022-08-01 12:52:58.374', [4, 15, 23, 32, 41, 48, 52, 68], 0], [

326 번째 시도:  [['2022-08-01 12:38:55.653', [8, 14, 26, 31, 40, 47, 60, 70], 0], ['2022-08-01 12:39:55.767', [8, 16, 26, 32, 40, 47, 61, 71], 0], ['2022-08-01 12:40:55.897', [9, 16, 26, 33, 41, 48, 62, 71], 0], ['2022-08-01 12:41:55.996', [9, 16, 27, 33, 41, 48, 63, 72], 0], ['2022-08-01 12:42:56.252', [9, 17, 27, 33, 41, 49, 63, 73], 0], ['2022-08-01 12:43:56.400', [10, 18, 27, 34, 43, 49, 63, 73], 0], ['2022-08-01 12:44:56.906', [11, 18, 28, 35, 44, 50, 63, 75], 0], ['2022-08-01 12:45:57.021', [11, 18, 28, 36, 45, 50, 65, 75], 0], ['2022-08-01 12:46:57.137', [11, 19, 29, 37, 45, 51, 65, 75], 0], ['2022-08-01 12:47:57.408', [12, 20, 29, 37, 46, 51, 65, 76], 0], ['2022-08-01 12:48:57.805', [13, 21, 29, 38, 46, 51, 66, 77], 0], ['2022-08-01 12:49:57.920', [13, 21, 30, 39, 47, 51, 66, 79], 0], ['2022-08-01 12:50:58.066', [1, 14, 21, 31, 40, 48, 52, 68, 79], 1], ['2022-08-01 12:51:58.228', [3, 15, 22, 31, 40, 48, 52, 68], 0], ['2022-08-01 12:52:58.374', [4, 15, 23, 32, 41, 48, 52, 68], 0], [

327 번째 시도:  [['2022-08-01 12:38:55.653', [8, 14, 26, 31, 40, 47, 60, 70], 0], ['2022-08-01 12:39:55.767', [8, 16, 26, 32, 40, 47, 61, 71], 0], ['2022-08-01 12:40:55.897', [9, 16, 26, 33, 41, 48, 62, 71], 0], ['2022-08-01 12:41:55.996', [9, 16, 27, 33, 41, 48, 63, 72], 0], ['2022-08-01 12:42:56.252', [9, 17, 27, 33, 41, 49, 63, 73], 0], ['2022-08-01 12:43:56.400', [10, 18, 27, 34, 43, 49, 63, 73], 0], ['2022-08-01 12:44:56.906', [11, 18, 28, 35, 44, 50, 63, 75], 0], ['2022-08-01 12:45:57.021', [11, 18, 28, 36, 45, 50, 65, 75], 0], ['2022-08-01 12:46:57.137', [11, 19, 29, 37, 45, 51, 65, 75], 0], ['2022-08-01 12:47:57.408', [12, 20, 29, 37, 46, 51, 65, 76], 0], ['2022-08-01 12:48:57.805', [13, 21, 29, 38, 46, 51, 66, 77], 0], ['2022-08-01 12:49:57.920', [13, 21, 30, 39, 47, 51, 66, 79], 0], ['2022-08-01 12:50:58.066', [1, 14, 21, 31, 40, 48, 52, 68, 79], 1], ['2022-08-01 12:51:58.228', [3, 15, 22, 31, 40, 48, 52, 68], 0], ['2022-08-01 12:52:58.374', [4, 15, 23, 32, 41, 48, 52, 68], 0], [

328 번째 시도:  [['2022-08-01 12:38:55.653', [8, 14, 26, 31, 40, 47, 60, 70], 0], ['2022-08-01 12:39:55.767', [8, 16, 26, 32, 40, 47, 61, 71], 0], ['2022-08-01 12:40:55.897', [9, 16, 26, 33, 41, 48, 62, 71], 0], ['2022-08-01 12:41:55.996', [9, 16, 27, 33, 41, 48, 63, 72], 0], ['2022-08-01 12:42:56.252', [9, 17, 27, 33, 41, 49, 63, 73], 0], ['2022-08-01 12:43:56.400', [10, 18, 27, 34, 43, 49, 63, 73], 0], ['2022-08-01 12:44:56.906', [11, 18, 28, 35, 44, 50, 63, 75], 0], ['2022-08-01 12:45:57.021', [11, 18, 28, 36, 45, 50, 65, 75], 0], ['2022-08-01 12:46:57.137', [11, 19, 29, 37, 45, 51, 65, 75], 0], ['2022-08-01 12:47:57.408', [12, 20, 29, 37, 46, 51, 65, 76], 0], ['2022-08-01 12:48:57.805', [13, 21, 29, 38, 46, 51, 66, 77], 0], ['2022-08-01 12:49:57.920', [13, 21, 30, 39, 47, 51, 66, 79], 0], ['2022-08-01 12:50:58.066', [1, 14, 21, 31, 40, 48, 52, 68, 79], 1], ['2022-08-01 12:51:58.228', [3, 15, 22, 31, 40, 48, 52, 68], 0], ['2022-08-01 12:52:58.374', [4, 15, 23, 32, 41, 48, 52, 68], 0], [

330 번째 시도:  [['2022-08-01 12:38:55.653', [8, 14, 26, 31, 40, 47, 60, 70], 0], ['2022-08-01 12:39:55.767', [8, 16, 26, 32, 40, 47, 61, 71], 0], ['2022-08-01 12:40:55.897', [9, 16, 26, 33, 41, 48, 62, 71], 0], ['2022-08-01 12:41:55.996', [9, 16, 27, 33, 41, 48, 63, 72], 0], ['2022-08-01 12:42:56.252', [9, 17, 27, 33, 41, 49, 63, 73], 0], ['2022-08-01 12:43:56.400', [10, 18, 27, 34, 43, 49, 63, 73], 0], ['2022-08-01 12:44:56.906', [11, 18, 28, 35, 44, 50, 63, 75], 0], ['2022-08-01 12:45:57.021', [11, 18, 28, 36, 45, 50, 65, 75], 0], ['2022-08-01 12:46:57.137', [11, 19, 29, 37, 45, 51, 65, 75], 0], ['2022-08-01 12:47:57.408', [12, 20, 29, 37, 46, 51, 65, 76], 0], ['2022-08-01 12:48:57.805', [13, 21, 29, 38, 46, 51, 66, 77], 0], ['2022-08-01 12:49:57.920', [13, 21, 30, 39, 47, 51, 66, 79], 0], ['2022-08-01 12:50:58.066', [1, 14, 21, 31, 40, 48, 52, 68, 79], 1], ['2022-08-01 12:51:58.228', [3, 15, 22, 31, 40, 48, 52, 68], 0], ['2022-08-01 12:52:58.374', [4, 15, 23, 32, 41, 48, 52, 68], 0], [

In [3]:
print("sdfsdf")

sdfsdf


In [4]:
def func(a,b):
    return a+b

print(func(3,3))

6
